# Extract Köppen-Geiger Climate Composition per NUTS-2 Region

In [2]:
import geopandas as gpd
import pandas as pd
from rasterstats import zonal_stats

import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt

# STEP 1: Load the NUTS-2 GeoJSON (High Resolution)

In [ ]:
import requests
import geopandas as gpd

# ----------------------------------------------------------------------------
# Download NUTS-2 GeoJSON directly from Eurostat GISCO
# ----------------------------------------------------------------------------

# Construct the URL using the GISCO API pattern
# Pattern: https://gisco-services.ec.europa.eu/distribution/v2/nuts/geojson/NUTS_RG_{SCALE}_{YEAR}_{PROJECTION}_LEVL_{LEVEL}.geojson

year = "2024"
scale = "03M"  # 1:3 Million (high resolution)
projection = "3035"  # EPSG:3035 (ETRS89-extended / LAEA Europe)
level = "2"  # NUTS level 2

url = f"https://gisco-services.ec.europa.eu/distribution/v2/nuts/geojson/NUTS_RG_{scale}_{year}_{projection}_LEVL_{level}.geojson"

print(f"Downloading from:\n{url}\n")

# Download the file
response = requests.get(url)

# Check if download was successful
if response.status_code == 200:
    # Save to local file
    output_filename = f"NUTS_RG_{scale}_{year}_{projection}_LEVL_{level}.geojson"
    with open(output_filename, 'wb') as f:
        f.write(response.content)
    print(f"✓ Download successful: {output_filename}")
    print(f"  File size: {len(response.content) / (1024*1024):.2f} MB")

    # Load directly into geopandas (optional: skip saving to disk)
    nuts_2 = gpd.read_file(output_filename)
    print(f"✓ Loaded {len(nuts_2)} NUTS-2 regions into GeoDataFrame")
    print(f"  CRS: {nuts_2.crs}")

else:
    print(f"✗ Download failed with status code: {response.status_code}")
    print(f"  Error: {response.text}")

https://gisco-services.ec.europa.eu/distribution/v2/nuts/geojson/NUTS_RG_03M_2024_3035_LEVL_2.geojson

✓ Download successful: NUTS_RG_03M_2024_3035_LEVL_2.geojson
  File size: 3.34 MB
✓ Loaded 299 NUTS-2 regions into GeoDataFrame
  CRS: EPSG:3035


# STEP 2: Load the Köppen-Geiger Raster and Check CRS

In [ ]:
koppen_raster_path = "koppen_geiger_0p00833333.tif"  # Your file

with rasterio.open(koppen_raster_path) as src:
    print(f"Raster CRS: {src.crs}")
    print(f"Raster bounds: {src.bounds}")
    print(f"Raster shape: {src.shape}")
    raster_crs = src.crs

# STEP 3: Reproject NUTS-2 to Match Raster CRS

In [ ]:
if nuts_2.crs != raster_crs:
    print(f"Reprojecting NUTS-2 from {nuts_2.crs} to {raster_crs}...")
    nuts_2 = nuts_2.to_crs(raster_crs)
    print("Reprojection complete.")

# STEP 4: Run CATEGORICAL Zonal Statistics

In [ ]:
print("Running zonal statistics (this may take 5-10 minutes for all of Europe)...")

stats = zonal_stats(
    nuts_2,
    koppen_raster_path,
    categorical=True,      # KEY: Returns counts per unique pixel value
    nodata=0,              # Treat 0 as "no data" (ocean/missing)
    all_touched=False      # Only count pixels whose CENTER is inside the polygon
)

print("Zonal statistics complete.")

# STEP 5: Convert Results to DataFrame and Normalize

In [ ]:
climate_df = pd.DataFrame(stats)

# Fill NaN with 0 (if a region has no pixels of a certain climate, count = 0)
climate_df = climate_df.fillna(0)

# Calculate total pixels per region
climate_df['total_pixels'] = climate_df.sum(axis=1)

# Normalize to percentages (0.0 to 1.0)
for col in climate_df.columns:
    if col != 'total_pixels' and col.replace('.0', '').isdigit():
        climate_class_num = col
        climate_df[f'pct_class_{climate_class_num}'] = (
            climate_df[climate_class_num] / climate_df['total_pixels']
        )

# Keep only the percentage columns
pct_cols = [c for c in climate_df.columns if c.startswith('pct_')]
climate_pct = climate_df[pct_cols].copy()


# STEP 6: Map Numeric Codes to Köppen Class Names

In [ ]:
koppen_legend = {
    1: 'Af', 2: 'Am', 3: 'Aw', 4: 'BWh', 5: 'BWk', 6: 'BSh', 7: 'BSk',
    8: 'Csa', 9: 'Csb', 10: 'Csc', 11: 'Cwa', 12: 'Cwb', 13: 'Cwc',
    14: 'Cfa', 15: 'Cfb', 16: 'Cfc', 17: 'Dsa', 18: 'Dsb', 19: 'Dsc',
    20: 'Dsd', 21: 'Dwa', 22: 'Dwb', 23: 'Dwc', 24: 'Dwd', 25: 'Dfa',
    26: 'Dfb', 27: 'Dfc', 28: 'Dfd', 29: 'ET', 30: 'EF'
}

# Rename columns from 'pct_class_8.0' to 'pct_Csa'
renamed_cols = {}
for col in pct_cols:
    # Extract the number: 'pct_class_8.0' -> 8
    num_str = col.replace('pct_class_', '').replace('.0', '')
    if num_str.isdigit():
        climate_code = int(num_str)
        if climate_code in koppen_legend:
            renamed_cols[col] = f'pct_{koppen_legend[climate_code]}'

climate_pct.rename(columns=renamed_cols, inplace=True)

# STEP 7: Merge Back to NUTS-2 GeoDataFrame

In [ ]:
final_gdf = pd.concat([
    nuts_2[['NUTS_ID', 'NUTS_NAME', 'CNTR_CODE', 'geometry']].reset_index(drop=True),
    climate_pct.reset_index(drop=True)
], axis=1)

# STEP 8: Save the Result

In [ ]:
final_gdf.drop(columns='geometry').to_csv('nuts2_koppen_climate_features.csv', index=False)

# Save as GeoPackage (with geometry, for mapping)
final_gdf.to_file('nuts2_koppen_climate_features.gpkg', driver='GPKG')

print("\n=== SUCCESS ===")
print("Output files created:")
print("  1. nuts2_koppen_climate_features.csv (for ML model)")
print("  2. nuts2_koppen_climate_features.gpkg (for mapping)")
print(f"\nSample of output:\n{final_gdf.head()}")

# STEP 9: Quick Validation Plot (Optional)

In [ ]:
example_region = final_gdf[final_gdf['NUTS_ID'] == 'ES51']  # Catalonia
if not example_region.empty:
    climate_cols = [c for c in example_region.columns if c.startswith('pct_')]
    climate_dist = example_region[climate_cols].iloc[0]
    climate_dist = climate_dist[climate_dist > 0.01]  # Only show >1%

    climate_dist.plot(kind='bar', title='Climate Composition: Catalonia (ES51)')
    plt.ylabel('Percentage of Region Area')
    plt.tight_layout()
    plt.savefig('example_catalonia_climate.png', dpi=150)
    print("\nValidation plot saved: example_catalonia_climate.png")